In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.special import sph_harm

In [3]:
##  fundamental constants

R_E = 6.378e6 # [m], radius of Earth
M_E = 5.974e24 # [kg], mass of Earth
M_sun = 1.989e30 # [kg], mass of Sun
AU = 1.4959e11 # [m], Earth-sun distanßßce
g = 9.81 # [m s-1], Earth acceleration due to gravity
bar = 1e5 # [Pa]
L_sun = 3.827e26 # [W], luminosity of sun
G = 6.673e-11 # [m3 kg-1 s-2], Gravitational Constant
sigma_SB = 5.67e-8 # Stefan-Boltzmann constant [W m-2 K-4]

## constants used in Salazar & Wordsworth, 2023

n = 2*np.pi/(224*60*60*24) # orbital mean motion [rad/s]
Cs_land = 1000000 # heat capacity of surface, [J kg-1]
cp = 1000 # specific heat of air, [J kg-1 K-1]
rho_mean = 5510 # density of planet, [kg m-3]
D = 1.66 # diffusivity constant 
R = 188 # specific gas constant, [J kg-1 K-1]
Cd = 0.0034 # surface drag coefficient
chi = 0.17 # reference thermal coupling factor
I = 0.4*M_E*R_E**2 # principle moment of inertia of planet
k2 = 0.23 # second order love number


In [7]:
def wind_speed(S, alpha, tau_sw, tau_lw, ps): # Us [m/s], Equation (23)
    k = tau_sw/tau_lw 
    F_bar = S*(1-alpha)*np.exp(-tau_sw)/np.pi # average incident stellar radiation at surface (Wm-2)
    SLW = S*(1-alpha)/8*(1+D/k - (1+D/k)*np.exp(-k*tau_lw)) # surface downwelling longwave (Wm-2), Equation (15)
    T_bar = np.power((F_bar+SLW)/sigma_SB, 1/4) # average surface temperature (K)
    T_eq = (S*(1-alpha)/(4*sigma_SB))**(1/4) # equilibrium surface temperature (K)
    Us = np.power(R/Cd * np.maximum((T_bar - T_eq),0.01) * (S/2)*(1-alpha)*np.exp(-tau_sw)*(1-np.exp(-tau_lw))/ps,1/3) 
    return(Us)

def qo_wo(S, alpha, tau_sw, tau_lw, ps): # output q_o (defined in Equation 29) and w_o
    k = tau_sw/tau_lw
    F_bar = S*(1-alpha)*np.exp(-tau_sw)/np.pi # average incident stellar radiation at surface (Wm-2)
    SLW = S*(1-alpha)/8*(1+D/k - (1+D/k)*np.exp(-k*tau_lw)) # surface downwelling longwave (Wm-2), Equation (15)
    T_bar = np.power((F_bar+SLW)/sigma_SB, 1/4) # average surface temperature (K)
    T_eq = (S*(1-alpha)/(4*sigma_SB))**(1/4) # equilibrium surface temperature (K)
    Us = wind_speed(S, alpha, tau_sw, tau_lw, ps)
    circ_strength = Us/Uso # circulation strength, Equation (22)
    delt_p = chi*ps*circ_strength # Equation (23)
    Cs = cp*delt_p/g + Cs_land # heat capacity of surface, including atmosphere
    w_o = 4*sigma_SB*T_bar**3/Cs # thermal equilibrium frequency
    qo = -(1/24)*np.sqrt(15/(2*np.pi))*S*(1-alpha)*np.exp(-tau_sw)*delt_p/(F_bar + SLW)
    return(qo, w_o)

def torque_analytic(S, alpha, tau_sw, tau_lw, ps, Omega, n): # q_tilde [Pa], Equation (29)
    qo, w_o = qo_wo(S, alpha, tau_sw, tau_lw, ps)
    if type(Omega) == str:
        sigma = w_o*1 # maximum thermal tide at Omega = w_o
    else:
        sigma = 2*(Omega-n)
    torque = -qo*sigma/w_o/(1+(sigma/w_o)**2) # Equation (29)
    return(torque)


def amp_analytic(S, alpha, tau_sw, tau_lw, ps, Omega, n): # q_norm [Pa], Equation (30)
    qo, w_o = qo_wo(S, alpha, tau_sw, tau_lw, ps)
    if type(Omega) == str:
        sigma = w_o*1 # maximum thermal tide at Omega = w_o
    else:
        sigma = 2*(Omega-n)
    amp = -qo/np.sqrt(1+(sigma/w_o)**2) # Equation (30)
    return(amp)


def lag_analytic(S, alpha, tau_sw, tau_lw, ps, Omega, n): # sin(2d), Equation (31)
    qo, w_o = qo_wo(S, alpha, tau_sw, tau_lw, ps)
    if type(Omega) == str:
        sigma = w_o*1 # maximum thermal tide at Omega = w_o
    else:
        sigma = 2*(Omega-n)
    lag = np.sin(-np.arctan(-sigma/w_o)) # Equation (31)
    return(lag)


def thermal_tide_torque(M, R_E, a,S, alpha, tau_sw, tau_lw, ps, Omega_list, n): # T_a, Equation (32)
    Ka = -(3/2)*3*M*R_E**3/((5*rho_mean*a**3)*np.sqrt(10/(3*np.pi)))
    q_tilde = torque_analytic(S, alpha, tau_sw, tau_lw, ps, Omega_list, n)
    return(-Ka*q_tilde)

def gravitational_tide_torque(M, R_E, a,Q,Qn, k2, Omega_list, n): # T_g, Equation (4)
    Kg = -(3/2)*G*M**2*R_E**5/a**6
    bg = k2/Q * np.sign(Omega_list-n) 
    return(Kg*bg)

def S_from_a(M, a):
    mu = np.log10(M/M_sun)
    L_star = L_sun * 10**(4.101*mu**3 + 8.162*mu**2 + 7.108*mu) # Equation (37)
    S = L_star/(4*np.pi*a**2) # stellar constant
    orbit_period = (a**3 * 4* np.pi**2/(G*M))**(1/2)
    return(S, 2*np.pi/orbit_period)

def eq_rotation_ana(M, R_E, a, alpha, tau_sw, tau_lw, ps, Q, Qn, k2): # equilibrium rotation rate, Equation (35)
    S, n = S_from_a(M,a)
    qo, w_o = qo_wo(S, alpha, tau_sw, tau_lw, ps)
    Ka = -(3/2)*3*M*R_E**3/((5*rho_mean*a**3)*np.sqrt(10/(3*np.pi)))
    Tg = gravitational_tide_torque(M, R_E, a,Q,Qn, k2, 2*n, n)
    B = -Tg/(Ka*qo)
    if 1 >= 4*B**2: 
        quad_form = (1 + np.sqrt(1-(4*B**2)))/(2*B) # stable equilibria
    else: # no asynchronous solution
        quad_form = 0 # tidally locked
    return(quad_form*w_o)

def critical_semimajor(S, M, R_E, alpha, tau_sw, tau_lw, ps, Q, Qn, k2):
    qo = qo_wo(S, alpha, tau_sw, tau_lw, ps)[0]
    return((10*np.pi/3)**(1/6)*np.power(G*M*R_E**2*k2*rho_mean/(qo*Q),1/3)/AU)
    
def critical_semimajor_leconte(M, R_E, ps, Q, Qn, k2):
    if ps == 10*bar:
        qo = 4080 # Table 1 (Leconte et al. 2015)
    elif ps == 1*bar:
        qo = 1180 # Table 1 (Leconte et al. 2015)
    return((10*np.pi/3)**(1/6)*np.power(G*M*R_E**2*k2*rho_mean/(qo*Q),1/3)/AU)
    
    

In [6]:
Uso = wind_speed(1137, 0.2, 0.00001, 1, 1*bar) # U_so [m/s]